In [ ]:
import os
import re
import pdfplumber
import pandas as pd
import camelot

In [ ]:
# for filename in os.listdir(pdf_dir):
#     if filename.lower().endswith(".pdf"):
#         pdf_path = os.path.join(pdf_dir, filename)
#         with pdfplumber.open(pdf_path) as pdf:
#             all_text = ""
#             preserved_layout_text = ""
#             body_lines = []
            
#             for page in pdf.pages:
#                 all_text += page.extract_text() + "\n" or ""
#                 # Use preserved_text with extra parameters to preserve spacing
#                 # preserved_text = "" 
#                 # preserved_text += page.extract_text(
#                 #     layout=True,      # Preserves layout including spaces
#                 #     keep_blank_chars=True,  # Keeps blank characters
#                 #     x_tolerance=1,    # Smaller tolerance preserves spacing better
#                 #     y_tolerance=1
#                 # ) or ""
#                 # preserved_layout_text += preserved_text + "\n"
#                 # print(f"-----All text:\n{all_text}\n-----")
#                 # print(f"-----Preserved Layout Text:\n{preserved_layout_text}\n-----")

#                 # BODY TEXT EXTRACTION
#                 lines = all_text.split('\n')
#                 # More precise header pattern that matches your document's header format
#                 header_pattern = re.compile(r'^\*\d{2}B\d+\*$')  # Pattern like "*05B00041981*"
#                 in_header = False
#                 for line in lines:
#                     # Check for header pattern
#                     if header_pattern.match(line.strip()):
#                         in_header = True
#                         continue
#                     # Skip the line immediately after header pattern
#                     if in_header and line.strip().startswith('Order '):
#                         in_header = False
#                         continue
#                     # Skip footer lines
#                     if re.match(r'^Page \d+ of \d+$', line.strip()):
#                         continue
#                     body_lines.append(line)
                
#                 body_text = '\n'.join(body_lines)
#                 # Remove any remaining blank lines at start/end
#                 body_text = body_text.strip()
#                 # print(f"{body_text}")

#                 # #EXTRACT ORDER DETAILS LINES
#                 order_details_lines = []
#                 capture = False
#                 for line in body_text.split('\n'):
#                     if "This document contains certified products. Please see table below for details." in line:
#                         capture = True
#                         continue
#                     if "Applicable Certifications" in line:
#                         capture = False
#                         continue
#                     if capture and line.strip():
#                         order_details_lines.append(line)
                
#                 # Join the order details lines and clean up
#                 order_details_text = '\n'.join(order_details_lines).strip()
#                 print(order_details_text)
                
#                 #Camelot extraction
#                 tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
#                 camelot_tables = []
#                 for table in tables:
#                     camelot_df = table.df
#                     camelot_tables.append(camelot_df)
#                     # print(f"----\n{camelot_df.to_string(index=False)}\n-----")

In [52]:
pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\Sample\Main"

for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)

        with pdfplumber.open(pdf_path) as pdf:
            all_text = ""           # Raw extracted text from all pages
            all_pages_body = []     # Store processed body text per page

            # Header pattern like "*05B00041981*"
            header_pattern = re.compile(r'^\*\d{2}B\d+\*$')

            # === PAGE LOOP ===
            for page in pdf.pages:
                # Append raw text to all_text (full PDF content, unprocessed)
                page_text = page.extract_text() or ""
                all_text += page_text + "\n"

                # Clean body text for this page
                lines = page_text.split('\n')
                in_header = False
                page_body_lines = []

                for line in lines:
                    if header_pattern.match(line.strip()):
                        in_header = True
                        continue
                    if in_header and line.strip().startswith('Order '):
                        in_header = False
                        continue
                    if re.match(r'^Page \d+ of \d+$', line.strip()):
                        continue
                    page_body_lines.append(line)

                cleaned_body = '\n'.join(page_body_lines).strip()
                all_pages_body.append(cleaned_body)

            # === Combine cleaned page texts ===
            body_text = "\n".join(all_pages_body)

            # === Camelot table extraction (run ONCE per file) ===
            tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
            camelot_tables = [table.df for table in tables]

            # === Extract order details ===
            order_details_lines = []
            capture = False

            for line in body_text.split("\n"):
                if "This document contains certified products. Please see table below for details." in line:
                    capture = True
                    continue
                if "Applicable Certifications" in line:
                    capture = False
                    continue
                if capture and line.strip():
                    order_details_lines.append(line)

            order_details_text = "\n".join(order_details_lines)

            # print(f"All Text:\n{all_text}")  # Raw text
            # print(f"Body Text:\n{body_text}")  # Cleaned text without headers/footers
            # print(f"Order Details:\n{order_details_text}")  # Extracted order details texts        
            # print(f"Tables extracted: {camelot_tables}")  # Number of tables extracted     


In [54]:
from babel.numbers import parse_decimal

# Capture each order block: from the first line until the line starting with Amount
order_blocks = re.findall(r'^(.*?Amount\s[\d\.,]+\s*\w*)', order_details_text, re.DOTALL | re.MULTILINE)

orders = []

for block in order_blocks:
    block = block.strip()
    lines = block.splitlines()
    print(f"Processing block:\n{block}\n")

    # Extract first word as Style No
    style_no = lines[0].split()[0] if lines else ""
    
    # Extract Amount value
    amount_match = re.search(r'Amount\s([\d\.,]+)', block)
    amount = amount_match.group(1) if amount_match else "0"

    # Convert European format to US float
    amount_usd = float(parse_decimal(amount, locale='de_DE'))

    # Store as dict
    orders.append({
        'Style No': style_no,
        'Amount USD': amount_usd
    })

# Convert to DataFrame
df = pd.DataFrame(orders)
print(df)


Processing block:
NORMA030 Womens Polo shortsleeve "GOTS + shipment FOB date 15. September 2025 Price 5,20
Fairtrade"
grey Customs no.: 6106 1000 00
INTERNAT1
Size XS S M L XL XXL 3XL 4XL 5XL
Qty 860 4.630 6.120 4.980 3.120 1.420 480 240 215
Total 22.065 PIECES
Amount 114.738,00 USD

Processing block:
NORMA031 Womens Polo lonsleeve "GOTS + shipment FOB date 15. September 2025 Price 5,70
Fairtrade"
grey Customs no.: 6106 1000 00
INTERNAT1
Size XS S M L XL XXL 3XL 4XL 5XL
Qty 1.070 4.400 5.720 3.730 2.170 1.390 440 220 160
Total 19.300 PIECES
Amount 110.010,00 USD

Processing block:
NORMA032 Womens Polo AZUBI shortsleeve shipment FOB date 15. September 2025 Price 5,20
"GOTS + Fairtrade"
grey Customs no.: 6106 1000 00
INTERNAT1
Size XS S M L XL XXL 3XL 4XL 5XL
Qty 320 1.240 1.890 1.480 600 340 160 70 55
Total 6.155 PIECES
Amount 32.006,00 USD

Processing block:
NORMA033 Womens Polo AZUBI longsleeve "GOTS shipment FOB date 15. September 2025 Price 5,70
+ Fairtrade"
grey Customs no.: 6106 1

In [ ]:

# Capture each order block: from the first line until the line starting with Amount
order_blocks = re.findall(r'^(.*?Amount\s[\d\.,]+\s*\w*)', order_details_text, re.DOTALL | re.MULTILINE)

orders = []

for block in order_blocks:
    block = block.strip()
    lines = block.splitlines()
    print(f"Processing block:\n{block}\n")

    # Extract first word as Style No
    style_no = lines[0].split()[0] if lines else ""
    
    # Extract Amount value
    amount_match = re.search(r'Amount\s([\d\.,]+)', block)
    amount = amount_match.group(1) if amount_match else "0"

    # Convert European format to US float
    amount_usd = float(parse_decimal(amount, locale='de_DE'))

    # Store as dict
    orders.append({
        'Style No': style_no
    })

# Convert to DataFrame
df = pd.DataFrame(orders)
print(df)


In [ ]:

# Capture each order block: from the first line until the line starting with Amount
order_blocks = re.findall(r'^(.*?Amount\s[\d\.,]+\s*\w*)', order_details_text, re.DOTALL | re.MULTILINE)

orders = []

for block in order_blocks:
    block = block.strip()
    lines = block.splitlines()
    print(f"Processing block:\n{block}\n")

    # Extract first word as Style No
    style_no = lines[0].split()[0] if lines else ""
    
    #Extract Customs no.
    customs_no_match = re.search(r'Customs\s+no\.?\s*:\s*([\d ]+)', block)
    if customs_no_match:
        customs_no = customs_no_match.group(1).strip()

    

    # Store as dict
    orders.append({
        'Style No': style_no,
        'Customs No': customs_no
        })

# Convert to DataFrame
df = pd.DataFrame(orders)
print(df)


Processing block:
NORMA030 Womens Polo shortsleeve "GOTS + shipment FOB date 15. September 2025 Price 5,20
Fairtrade"
grey Customs no.: 6106 1000 00
INTERNAT1
Size XS S M L XL XXL 3XL 4XL 5XL
Qty 860 4.630 6.120 4.980 3.120 1.420 480 240 215
Total 22.065 PIECES
Amount 114.738,00 USD

Processing block:
NORMA031 Womens Polo lonsleeve "GOTS + shipment FOB date 15. September 2025 Price 5,70
Fairtrade"
grey Customs no.: 6106 1000 00
INTERNAT1
Size XS S M L XL XXL 3XL 4XL 5XL
Qty 1.070 4.400 5.720 3.730 2.170 1.390 440 220 160
Total 19.300 PIECES
Amount 110.010,00 USD

Processing block:
NORMA032 Womens Polo AZUBI shortsleeve shipment FOB date 15. September 2025 Price 5,20
"GOTS + Fairtrade"
grey Customs no.: 6106 1000 00
INTERNAT1
Size XS S M L XL XXL 3XL 4XL 5XL
Qty 320 1.240 1.890 1.480 600 340 160 70 55
Total 6.155 PIECES
Amount 32.006,00 USD

Processing block:
NORMA033 Womens Polo AZUBI longsleeve "GOTS shipment FOB date 15. September 2025 Price 5,70
+ Fairtrade"
grey Customs no.: 6106 1